In [2]:
import pandas as pd
from streaming_optimizer import optimize_streaming_packages
from datetime import datetime



In [47]:
games_raw = pd.read_csv('data/bc_game.csv') # some games and when they are played
streaming_packages_raw = pd.read_csv('data/bc_streaming_package.csv') # streaming packages with price
streaming_offers_raw = pd.read_csv('data/bc_streaming_offer.csv')  # which service offers which game, live? on-demand-highlights?

package_offers = streaming_packages_raw.merge(
    streaming_offers_raw, left_on='id', right_on='streaming_package_id', how='right')

merged = games_raw.merge(package_offers, left_on='id', right_on='game_id', how='right'
                         , suffixes=('_game', '_offer'))
merged[merged['team_home'] == 'Bologna FC']
merged[merged['id_game'] == 8548]

,id_game,team_home,team_away,starts_at,tournament_name,id_offer,name,monthly_price_cents,monthly_price_yearly_subscription_in_cents,game_id,streaming_package_id,live,highlights
6934,1638,Bologna FC,AC Mailand,2023-08-21 18:45:00,Serie A 23/24,2,MagentaTV - MegaSport,NaN,6000,1638,2,1,1
6935,1638,Bologna FC,AC Mailand,2023-08-21 18:45:00,Serie A 23/24,18,DAZN - Unlimited,4499.0,3499,1638,18,1,1
6936,1638,Bologna FC,AC Mailand,2023-08-21 18:45:00,Serie A 23/24,38,DAZN - Super Sports,2499.0,1999,1638,38,1,1
6937,1638,Bologna FC,AC Mailand,2023-08-21 18:45:00,Serie A 23/24,53,waipu.tv - Perfect Plus DAZN,5499.0,4499,1638,53,1,1
6986,1651,Bologna FC,Cagliari Calcio,2023-09-02 16:30:00,Serie A 23/24,2,MagentaTV - MegaSport,NaN,6000,1651,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31210,8522,Bologna FC,OSC Lille,2024-11-27 20:00:00,UEFA Champions League 24/25,52,waipu.tv - Perfect Plus,1299.0,1299,8522,52,0,1
31211,8522,Bologna FC,OSC Lille,2024-11-27 20:00:00,UEFA Champions League 24/25,53,waipu.tv - Perfect Plus DAZN,5499.0,4499,8522,53,1,1
31212,8522,Bologna FC,OSC Lille,2024-11-27 20:00:00,UEFA Champions League 24/25,54,O2 - TV M,699.0,699,8522,54,0,1
31213,8522,Bologna FC,OSC Lille,2024-11-27 20:00:00,UEFA Champions League 24/25,55,O2 - TV M mit Pay-TV,1398.0,1398,8522,55,0,1


In [42]:
game_ids_of_interest = games_raw[(games_raw['team_home'] == 'Bologna FC')]['id'].unique() #list(range(31, 39)) # Input from the FE

In [44]:
games_raw[(games_raw['team_home'] == 'Bologna FC') ]

,id,team_home,team_away,starts_at,tournament_name
1635,1638,Bologna FC,AC Mailand,2023-08-21 18:45:00,Serie A 23/24
1648,1651,Bologna FC,Cagliari Calcio,2023-09-02 16:30:00,Serie A 23/24
1674,1677,Bologna FC,SSC Neapel,2023-09-24 16:00:00,Serie A 23/24
1689,1692,Bologna FC,FC Empoli,2023-10-01 10:30:00,Serie A 23/24
1710,1713,Bologna FC,Frosinone Calcio,2023-10-22 13:00:00,Serie A 23/24
1726,1729,Bologna FC,Lazio Rom,2023-11-03 19:45:00,Serie A 23/24
1755,1758,Bologna FC,FC Turin,2023-11-27 19:45:00,Serie A 23/24
1783,1786,Bologna FC,AS Rom,2023-12-17 17:00:00,Serie A 23/24
1791,1794,Bologna FC,Atalanta BC,2023-12-23 14:00:00,Serie A 23/24
1806,1809,Bologna FC,Genua CFC,2024-01-05 19:45:00,Serie A 23/24


In [31]:
## Preprocess data (make everthing as small as possible)

### Filter packages to include only relevant ones
# Identify relevant package IDs from offers
relevant_package_ids = streaming_offers_raw[streaming_offers_raw['game_id'].isin(game_ids_of_interest)]['streaming_package_id'].unique()

# Filter packages to include only relevant ones
filtered_packages = streaming_packages_raw[streaming_packages_raw['id'].isin(relevant_package_ids)]

filtered_packages['yearly_price'] = filtered_packages['monthly_price_yearly_subscription_in_cents'] * 12
filtered_packages = filtered_packages.drop(columns=['monthly_price_yearly_subscription_in_cents'])


games = games_raw[games_raw['id'].isin(game_ids_of_interest)]

# Filter offers to include only the games of interest
filtered_offers = streaming_offers_raw[streaming_offers_raw['game_id'].isin(game_ids_of_interest)]


# Extract unique package IDs
packages = filtered_packages['id'].unique().tolist()

# Create game_dates dictionary
games.loc[:, 'starts_at'] = pd.to_datetime(games['starts_at'])
game_dates = games.set_index('id')['starts_at'].apply(lambda x: x.date()).to_dict()


# Create C_month and C_year dictionaries, dropping packages with NA for the respective type
C_month = filtered_packages.dropna(subset=['monthly_price_cents']) \
    .set_index('id')['monthly_price_cents'].to_dict()
C_year = filtered_packages.dropna(subset=['yearly_price']) \
    .set_index('id')['yearly_price'].to_dict()


# Create P_g dictionary
P_g = filtered_offers.groupby('game_id')['streaming_package_id'].apply(list).to_dict()

results = optimize_streaming_packages(packages, game_ids_of_interest, game_dates, C_month, C_year, P_g)

if results is not None:
    # Print results
    print("Status:", results["status"])
    print("Total Cost:", results["total_cost"])
    print("Active Monthly Subscriptions:")
    for sub in results["active_monthly_subscriptions"]:
        print(f"  Package: {sub['package']}, Start Date: {sub['start_date'].strftime('%Y-%m-%d')}")
    print("Active Yearly Subscriptions:")
    for sub in results["active_yearly_subscriptions"]:
        print(f"  Package: {sub['package']}, Start Date: {sub['start_date'].strftime('%Y-%m-%d')}")
else:
    print("No results returned from optimize_streaming_packages function.")

C:\Users\tmari\AppData\Local\Temp\ipykernel_27356\7461027.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_packages['yearly_price'] = filtered_packages['monthly_price_yearly_subscription_in_cents'] * 12


KeyError: np.int64(8548)

In [52]:
games_with_no_offers = []

for game_id in game_ids_of_interest:
    if game_id not in P_g:
        games_with_no_offers.append(game_id)

game_ids_of_interest = [game_id for game_id in game_ids_of_interest if game_id in P_g]

# Print games with no offers in a nicer format
print("Games with no offers:")
for game_id in games_with_no_offers:
    game_details = games[games['id'] == game_id].iloc[0]
    print(f"Game ID: {game_id}, Home Team: {game_details['team_home']}, Away Team: {game_details['team_away']}, Date: {game_details['starts_at']}")


Games with no offers: [np.int64(8548)]


In [7]:
from streaming_optimizer import optimize_streaming_packages
from datetime import datetime, timedelta

# Example Input Data
packages = ["P1", "P2"]
games = ["G1", "G2", "G3", "G4", "G5", 'G6', 'G7']
game_dates = {
    "G1": datetime(2023, 1, 15),
    "G2": datetime(2023, 2, 20),
    "G3": datetime(2023, 3, 25),
    "G4": datetime(2023, 4, 10),
    "G5": datetime(2023, 4, 20),
    "G6": datetime(2024, 1, 10),
    "G7": datetime(2024, 3, 15),
}
C_month = {'P1': 20}
C_year = {'P1': 180}
P_g = {'G1': ['P1'], 'G2': ['P1'], 'G3': ['P1', 'P2'], 'G4': ['P1', 'P2'], 'G5': ['P1', 'P2'],
    'G6': ['P1'], 'G7': ['P1', 'P2']}
results = optimize_streaming_packages(packages, games, game_dates, C_month, C_year, P_g)


if results is not None:
    # Print results
    print("Status:", results["status"])
    print("Total Cost:", results["total_cost"])
    print("Active Monthly Subscriptions:")
    for sub in results["active_monthly_subscriptions"]:
        print(f"  Package: {sub['package']}, Start Date: {sub['start_date'].strftime('%Y-%m-%d')}")
    print("Active Yearly Subscriptions:")
    for sub in results["active_yearly_subscriptions"]:
        print(f"  Package: {sub['package']}, Start Date: {sub['start_date'].strftime('%Y-%m-%d')}")
else:
    print("No results returned from optimize_streaming_packages function.")



Status: Optimal
Total Cost: 100.0
Active Monthly Subscriptions:
  Package: P1, Start Date: 2023-01-15
  Package: P1, Start Date: 2023-02-20
  Package: P1, Start Date: 2023-03-25
  Package: P1, Start Date: 2024-01-10
  Package: P1, Start Date: 2024-03-15
Active Yearly Subscriptions:
